In [1]:
# import all necessary modules
from oauth2client.service_account import ServiceAccountCredentials
import httplib2
import os
from apiclient import discovery
import oauth2client
from oauth2client import client
from oauth2client import tools

In [ ]:
%ls

In [3]:
# create service account credential
credentials = ServiceAccountCredentials.from_json_keyfile_name('key-file.json', 
                                                               scopes='https://www.googleapis.com/auth/drive')

In [4]:
# authorize client
http = credentials.authorize(httplib2.Http())

In [5]:
# build a service object
service = discovery.build('drive', 'v3', http=http)

In [6]:
# list all files
data = service.files().list(fields='nextPageToken,files(name,id,kind,mimeType,parents,permissions,shared)').execute()

In [7]:
# view all files
import pandas as pd
pd.DataFrame(data['files']).head(5)

,id,kind,mimeType,name,parents,permissions,shared
0,0B_fFUcPD3iZKZF9TajJIUno4OW8,drive#file,text/plain,data.txt,[0APfFUcPD3iZKUk9PVA],"[{u'kind': u'drive#permission', u'displayName'...",False
1,0B_fFUcPD3iZKRmJsaHgtOTBwVUE,drive#file,text/plain,data.txt,[0APfFUcPD3iZKUk9PVA],"[{u'kind': u'drive#permission', u'displayName'...",False
2,0B_fFUcPD3iZKNzgtWnlDNmp4TE0,drive#file,application/vnd.google-apps.folder,gdrive-test-folder,[0APfFUcPD3iZKUk9PVA],"[{u'kind': u'drive#permission', u'displayName'...",False
3,0B_fFUcPD3iZKYkhVdS1nTnZ3QTQ,drive#file,application/vnd.google-apps.folder,gdrive-test-folder,[0APfFUcPD3iZKUk9PVA],"[{u'kind': u'drive#permission', u'displayName'...",False
4,0B_fFUcPD3iZKeXNGUlQzZnhWM1k,drive#file,application/vnd.google-apps.folder,gdrive-test-folder,[0APfFUcPD3iZKUk9PVA],"[{u'kind': u'drive#permission', u'displayName'...",False


In [8]:
# create a folder
def create_folder(folder_name, parent_id):
    # if parent is not specified not folder would be created 
    # inside root
    metadata = {
        'name': folder_name,
        'mimeType': 'application/vnd.google-apps.folder',
    }
    if parent_id:
        metadata['parents'] = [parent_id]
    
    folder = service.files().create(
        body=metadata,
        fields='id'
    ).execute()
    return folder['id']
ID = create_folder('gdrive-test-folder', None)
print("New Folder ID:" + ID)
# Get file metadata
service.files().get(fileId=ID, fields="name,id,mimeType,parents").execute()

New Folder ID:0B_fFUcPD3iZKNDNKLXpQUktPOWs


{u'id': u'0B_fFUcPD3iZKNDNKLXpQUktPOWs',
 u'mimeType': u'application/vnd.google-apps.folder',
 u'name': u'gdrive-test-folder',
 u'parents': [u'0APfFUcPD3iZKUk9PVA']}

In [9]:
# delete the file/folder that we have created recently
service.files().delete(fileId=ID, fields="id").execute()

''

In [10]:
service.files().list(q="name='gdrive-test-folder'", fields="files(id,name)").execute()

{u'files': [{u'id': u'0B_fFUcPD3iZKNzgtWnlDNmp4TE0',
   u'name': u'gdrive-test-folder'},
  {u'id': u'0B_fFUcPD3iZKYkhVdS1nTnZ3QTQ', u'name': u'gdrive-test-folder'},
  {u'id': u'0B_fFUcPD3iZKeXNGUlQzZnhWM1k', u'name': u'gdrive-test-folder'},
  {u'id': u'0B_fFUcPD3iZKRmNkc3R0ajREb2M', u'name': u'gdrive-test-folder'},
  {u'id': u'0B_fFUcPD3iZKaEE3aUMyakdrQUU', u'name': u'gdrive-test-folder'},
  {u'id': u'0B_fFUcPD3iZKeWstUE1tS05BME0', u'name': u'gdrive-test-folder'},
  {u'id': u'0B_fFUcPD3iZKWWlaZFJpM08zTlE', u'name': u'gdrive-test-folder'},
  {u'id': u'0B_fFUcPD3iZKNzVQMFVtOUdpMEU', u'name': u'gdrive-test-folder'}]}

In [11]:
# upload a file with MedialUpload
from googleapiclient.http import MediaFileUpload
media = MediaFileUpload(
    'data.txt', # filename,
    mimetype="text/plain",
    resumable=True
)
body = {
    'name': 'data.txt',
    'mimeType': 'text/plain'
}
response = service.files().create(
    body=body,
    media_body=media,
    fields="id"
).execute()
ID = response['id']

In [12]:
# share file/folder with other user
permission_id = service.permissions().create(
    fileId=ID,
    body={
        "type": "user",
        "role": "writer",
        "emailAddress": "salam@datalicious.com"
    },
    fields='id'
).execute()